# Infrastructure: Simple API Management

Sets up the infrastructure for a simple API Management setup with a publicly-accessible APIM instance.

⌚ **Expected *Run All* runtime: ~3 minutes**

### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment. 

❗️ **Modify entries under _1) User-defined parameters_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 3
apim_sku    = APIM_SKU.BASICV2
deployment  = INFRASTRUCTURE.SIMPLE_APIM

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)

# 3) Define the APIs and their operations and policies

# Policies
hello_world_policy_xml = utils.read_policy_xml(HELLO_WORLD_XML_POLICY_PATH)

# Hello World (Root)
api_hwroot_get = GET_APIOperation('This is a GET for API 1', hello_world_policy_xml)
api_hwroot = API('hello-world', 'Hello World', '', 'This is the root API for Hello World', operations = [api_hwroot_get])

# APIs Array
apis: List[API] = [api_hwroot]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apimSku'   : {'value': apim_sku.value},
    'apis'      : {'value': [api.to_dict() for api in apis]}
}

# 2) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 3) Check the deployment outputs
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

### ✅ 3. Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

In [ ]:
from apimrequests import ApimRequests

reqs = ApimRequests(apim_gateway_url)

output = reqs.singleGet('/', msg = 'Calling Hello World (Root) API')

utils.print_ok('All done!')

### 🗑️ Clean up resources

When you're finished experimenting, it's advisable to remove all associated resources from Azure to avoid unnecessary cost.
Use the [clean-up notebook](clean-up.ipynb) for that.